# Plot the Amtrak Thruway routes

Do these make sense?

Think about how to grab local buses that also travel the same origin / destination.

In [ ]:
import geopandas as gpd
import pandas as pd

import _utils
from _utils import GCS_FILE_PATH, SELECTED_DATE

In [ ]:
trip_gdf = gpd.read_parquet(
    f"{GCS_FILE_PATH}amtrak_thruway_trips_{SELECTED_DATE}.parquet")

gdf = _utils.aggregate_to_route(trip_gdf)

In [ ]:
gdf = gdf.assign(
    route_length = gdf.geometry.to_crs("EPSG:2229").length / 5280
)

In [ ]:
# Use a cut-off and filter down CA transit routes of at least 50 mi long
gdf.route_length.describe()

In [ ]:
print(gdf.route_type.value_counts())
print(gdf.route_id.value_counts())

In [ ]:
def make_map(gdf: gpd.GeoDataFrame, route: str): 
    
    subset = gdf[gdf.route_id==route].reset_index(drop=True)
    
    print(f"Route: {route}    Route Type: {subset.route_type.iloc[0]}")

    display(subset[["origin_stop_name", "destination_stop_name"]].drop_duplicates())
    
    m = subset.explore("trip_id_nunique", categorical = False)
    display(m)
    
    display(subset)


In [ ]:
amtrak_routes = gdf[gdf.route_type=='3'].route_id.unique().tolist()

for r in amtrak_routes:
    make_map(gdf, r)

In [ ]:
amtrak_rail_routes = gdf[gdf.route_type=='2'].route_id.unique().tolist()

for r in amtrak_rail_routes:
    make_map(gdf, r)

In [ ]:
# https://www.rome2rio.com/map/San-Jose/Stockton#r/Train/s/0

* MST runs longer routes
* Eastern Sierra Transit (Lancaster to Reno)
* Sonoma County Transit...or is it Golden Gate Transit 
* SolTrans

Also maybe grab a full week to get all the unique OD pairs

1. Follow Amtrak thruway (or same OD)
2. Don't follow Amtrak thruway but go longer distance
Longer route (> certain distance) get at Eastern Sierra
3. Long routes that travel on SHN on extended period of time


Use `pd.get_dummies` to find OD pairs

Also, wrangle df to be long, and then you can do the dissolve for OD buffers.


Draw buffer around rail lines, see which bus routes have significant overlay 